In [28]:
import pandas as pd
import numpy as np
import random as rnd
import os
import datetime
import pickle

from scipy import stats
from scipy.special import boxcox1p

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm_notebook

from itertools import product

import gc

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone

import xgboost as xgb
import lightgbm as lgb

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error,r2_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold

from imblearn.under_sampling import RandomUnderSampler

pd.set_option('display.max_rows', 600)
pd.set_option('display.max_columns', 200)

import warnings
warnings.filterwarnings('ignore')

In [29]:
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

In [30]:
meta_train = pd.read_csv('../input/metadata_train.csv')
meta_test = pd.read_csv('../input/metadata_test.csv')

In [31]:
numpeaks = np.load('../features/numpeaks.npy')
numpospeaks = np.load('../features/numpospeaks.npy')
numnegpeaks = np.load('../features/numnegpeaks.npy')

maxpeakwidth = np.load('../features/maxpeakwidth.npy')
minpeakwidth = np.load('../features/minpeakwidth.npy')
meanpeakwidth = np.load('../features/meanpeakwidth.npy')
maxamp = np.load('../features/maxamp.npy')
minamp = np.load('../features/minamp.npy')
meanamp = np.load('../features/meanamp.npy')

stdampall = np.load('../features/stdampall.npy')
stdposall = np.load('../features/stdposall.npy')
stdwidthall = np.load('../features/stdwidthall.npy')

stdampq2 = np.load('../features/stdampq2.npy')
stdposq2 = np.load('../features/stdposq2.npy')
stdwidthq2 = np.load('../features/stdwidthq2.npy')
stdampq3 = np.load('../features/stdampq3.npy')
stdposq3 = np.load('../features/stdposq3.npy')
stdwidthq3 = np.load('../features/stdwidthq3.npy')
stdampq4 = np.load('../features/stdampq4.npy')
stdposq4 = np.load('../features/stdposq4.npy')
stdwidthq4 = np.load('../features/stdwidthq4.npy')


In [32]:
train_features = pd.DataFrame({'NumPeaks':numpeaks, 'NumPosPeaks':numpospeaks, 'NumNegPeaks':numnegpeaks, 'MaxWidth':maxpeakwidth,
                         'MinWidth':minpeakwidth, 'MeanWidth':meanpeakwidth, 'MaxAmp':maxamp, 'MinAmp':minamp, 'MeanAmp':meanamp,
                        'StdAmpAll':stdampall, 'StdPosAll':stdposall, 'StdWidthAll':stdwidthall, 'StdAmpQ2':stdampq2, 'StdPosQ2':stdposq2,
                         'StdWidthQ2':stdwidthq2, 'StdAmpQ3':stdampq3, 'StdPosQ3':stdposq3, 'StdWidthQ3':stdwidthq3,
                        'StdAmpQ4':stdampq4, 'StdPosQ4':stdposq4, 'StdWidthQ4':stdwidthq4,})

In [33]:
train = pd.concat([meta_train, train_features], axis=1)

In [34]:
train.head()

,signal_id,id_measurement,phase,target,NumPeaks,NumPosPeaks,NumNegPeaks,MaxWidth,MinWidth,MeanWidth,MaxAmp,MinAmp,MeanAmp,StdAmpAll,StdPosAll,StdWidthAll,StdAmpQ2,StdPosQ2,StdWidthQ2,StdAmpQ3,StdPosQ3,StdWidthQ3,StdAmpQ4,StdPosQ4,StdWidthQ4
0,0,0,0,0,105,48,57,41741,1,3676.451923,46.676929,-21.931899,-0.000125,11.866878,77424.493902,7099.857490,11.214203,25011.751218,6165.933632,12.479814,22686.469829,2801.657114,8.081718,22092.125444,8984.882428
1,1,0,1,0,23,14,9,69734,1,10977.954545,13.454715,-12.666357,0.000094,8.663170,76017.661321,16116.212440,9.545417,8608.000000,0.000000,8.286657,19963.709475,6986.037612,5.719954,15027.735166,9284.827461
2,2,0,2,0,121,67,54,58988,1,3175.458333,26.176798,-30.415865,0.000112,11.285462,74713.050192,7570.433485,14.920124,24129.371090,7265.204158,11.041417,20894.197560,1152.704984,11.427326,21303.210731,5779.776683
3,3,1,0,1,217,99,118,44999,1,1820.212963,91.147620,-38.193195,0.000239,11.473998,131660.062871,4198.542517,5.535926,25928.722290,12482.994099,5.538974,22936.531117,3615.659712,5.086507,34060.095196,2594.629391
4,4,1,1,1,141,68,73,37017,1,2803.257143,14.246623,-9.247347,-0.000065,4.289404,126355.390058,5433.900865,4.720264,21722.232143,7837.139326,3.000449,25632.115546,9773.675906,3.425179,29998.890319,4142.901139


In [35]:
signal_id = train['signal_id'].values
del train['signal_id']

In [36]:
target = train['target'].values
del train['target']

In [37]:
from sklearn.metrics import matthews_corrcoef

In [38]:
skf = StratifiedKFold(n_splits=4, random_state=123, shuffle=True)

In [39]:
target

array([0, 0, 0, ..., 0, 0, 0])

In [41]:
random_state = [11, 22, 33, 44]

lgb_params = {
               'feature_fraction': 0.8,
               'metric': 'binary_logloss',
               'nthread':8, 
               'learning_rate': 0.1, 
               'objective': 'binary', 
               'num_leaves': 2**4,
               'verbose':0, 
               'seed':123
              }
clf = []
train.fillna(0, inplace=True)

for state in random_state:
    sampler = RandomUnderSampler(random_state=state)
    train_re, target_re = sampler.fit_resample(train, target)
    model_lgb = lgb.train(lgb_params, lgb.Dataset(train_re, label=target_re), 80)
    clf.append(model_lgb)

In [42]:
numpeaks = np.load('../features/numpeaks_test.npy')
numpospeaks = np.load('../features/numpospeaks_test.npy')
numnegpeaks = np.load('../features/numnegpeaks_test.npy')

maxpeakwidth = np.load('../features/maxpeakwidth_test.npy')
minpeakwidth = np.load('../features/minpeakwidth_test.npy')
meanpeakwidth = np.load('../features/meanpeakwidth_test.npy')
maxamp = np.load('../features/maxamp_test.npy')
minamp = np.load('../features/minamp_test.npy')
meanamp = np.load('../features/meanamp_test.npy')

stdampall = np.load('../features/stdampall_test.npy')
stdposall = np.load('../features/stdposall_test.npy')
stdwidthall = np.load('../features/stdwidthall_test.npy')

stdampq2 = np.load('../features/stdampq2_test.npy')
stdposq2 = np.load('../features/stdposq2_test.npy')
stdwidthq2 = np.load('../features/stdwidthq2_test.npy')
stdampq3 = np.load('../features/stdampq3_test.npy')
stdposq3 = np.load('../features/stdposq3_test.npy')
stdwidthq3 = np.load('../features/stdwidthq3_test.npy')
stdampq4 = np.load('../features/stdampq4_test.npy')
stdposq4 = np.load('../features/stdposq4_test.npy')
stdwidthq4 = np.load('../features/stdwidthq4_test.npy')


test_features = pd.DataFrame({'NumPeaks':numpeaks, 'NumPosPeaks':numpospeaks, 'NumNegPeaks':numnegpeaks, 'MaxWidth':maxpeakwidth,
                         'MinWidth':minpeakwidth, 'MeanWidth':meanpeakwidth, 'MaxAmp':maxamp, 'MinAmp':minamp, 'MeanAmp':meanamp,
                        'StdAmpAll':stdampall, 'StdPosAll':stdposall, 'StdWidthAll':stdwidthall, 'StdAmpQ2':stdampq2, 'StdPosQ2':stdposq2,
                         'StdWidthQ2':stdwidthq2, 'StdAmpQ3':stdampq3, 'StdPosQ3':stdposq3, 'StdWidthQ3':stdwidthq3,
                        'StdAmpQ4':stdampq4, 'StdPosQ4':stdposq4, 'StdWidthQ4':stdwidthq4,})

In [43]:
test = pd.concat([meta_test, test_features], axis=1)

In [44]:
test_id = test['signal_id'].values
del test['signal_id']

In [45]:
preds = None
for model in clf:
    if preds is None:
        preds = model.predict(test)
    else:
        preds += model.predict(test)
    
preds = preds/4

In [46]:
submission = pd.DataFrame({'signal_id':test_id,'target':(preds>0.5).astype(int)})

In [48]:
submission['target'].value_counts()

0    17460
1     2877
Name: target, dtype: int64

In [49]:
submission.to_csv('../output/submission_2.csv', index=False)

In [22]:
sampler = RandomUnderSampler(random_state=42)
train_re, target_re = sampler.fit_resample(train.fillna(0), target)

In [24]:
train_re.shape

(1050, 23)

In [31]:
pd.Series(target_re).value_counts()

1    525
0    525
dtype: int64

In [33]:
clf = []
val_pred = pd.DataFrame([])
val_mcc = []
lgb_params = {
               'feature_fraction': 0.8,
               'metric': 'binary_logloss',
               'nthread':8, 
               'learning_rate': 0.1, 
               'objective': 'binary', 
               'num_leaves': 2**4,
               'verbose':0, 
               'seed':123
              }

for train_idx, val_idx in skf.split(train_re, target_re):
    X_tr, y_tr = train_re[train_idx, :], target_re[train_idx]
    X_val, y_val = train_re[val_idx, :], target_re[val_idx]
 
    model_lgb = lgb.train(lgb_params, lgb.Dataset(X_tr, label=y_tr), 500,\
                           valid_sets=lgb.Dataset(X_val, label=y_val), early_stopping_rounds=30)
    pred_lgb = model_lgb.predict(X_val)
    pred_lgb = pd.DataFrame(pred_lgb, index=val_idx)
    
    val_pred = pd.concat([val_pred, pred_lgb], axis=1)
    val_mcc.append(matthews_corrcoef(y_val, (pred_lgb>0.5).astype(int)))
    clf.append(model_lgb)

[1]	valid_0's binary_logloss: 0.638708
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's binary_logloss: 0.593258
[3]	valid_0's binary_logloss: 0.556483
[4]	valid_0's binary_logloss: 0.526723
[5]	valid_0's binary_logloss: 0.501264
[6]	valid_0's binary_logloss: 0.481347
[7]	valid_0's binary_logloss: 0.46333
[8]	valid_0's binary_logloss: 0.448833
[9]	valid_0's binary_logloss: 0.433952
[10]	valid_0's binary_logloss: 0.420137
[11]	valid_0's binary_logloss: 0.41113
[12]	valid_0's binary_logloss: 0.404058
[13]	valid_0's binary_logloss: 0.394153
[14]	valid_0's binary_logloss: 0.386149
[15]	valid_0's binary_logloss: 0.38148
[16]	valid_0's binary_logloss: 0.376378
[17]	valid_0's binary_logloss: 0.369567
[18]	valid_0's binary_logloss: 0.366851
[19]	valid_0's binary_logloss: 0.366295
[20]	valid_0's binary_logloss: 0.365599
[21]	valid_0's binary_logloss: 0.364225
[22]	valid_0's binary_logloss: 0.363502
[23]	valid_0's binary_logloss: 0.363138
[24]	valid_0's binary_logloss:

In [24]:
train_re.shape

(1050, 23)

In [31]:
pd.Series(target_re).value_counts()

1    525
0    525
dtype: int64

In [33]:
clf = []
val_pred = pd.DataFrame([])
val_mcc = []
lgb_params = {
               'feature_fraction': 0.8,
               'metric': 'binary_logloss',
               'nthread':8, 
               'learning_rate': 0.1, 
               'objective': 'binary', 
               'num_leaves': 2**4,
               'verbose':0, 
               'seed':123
              }

for train_idx, val_idx in skf.split(train_re, target_re):
    X_tr, y_tr = train_re[train_idx, :], target_re[train_idx]
    X_val, y_val = train_re[val_idx, :], target_re[val_idx]
 
    model_lgb = lgb.train(lgb_params, lgb.Dataset(X_tr, label=y_tr), 500,\
                           valid_sets=lgb.Dataset(X_val, label=y_val), early_stopping_rounds=30)
    pred_lgb = model_lgb.predict(X_val)
    pred_lgb = pd.DataFrame(pred_lgb, index=val_idx)
    
    val_pred = pd.concat([val_pred, pred_lgb], axis=1)
    val_mcc.append(matthews_corrcoef(y_val, (pred_lgb>0.5).astype(int)))
    clf.append(model_lgb)

[1]	valid_0's binary_logloss: 0.638708
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's binary_logloss: 0.593258
[3]	valid_0's binary_logloss: 0.556483
[4]	valid_0's binary_logloss: 0.526723
[5]	valid_0's binary_logloss: 0.501264
[6]	valid_0's binary_logloss: 0.481347
[7]	valid_0's binary_logloss: 0.46333
[8]	valid_0's binary_logloss: 0.448833
[9]	valid_0's binary_logloss: 0.433952
[10]	valid_0's binary_logloss: 0.420137
[11]	valid_0's binary_logloss: 0.41113
[12]	valid_0's binary_logloss: 0.404058
[13]	valid_0's binary_logloss: 0.394153
[14]	valid_0's binary_logloss: 0.386149
[15]	valid_0's binary_logloss: 0.38148
[16]	valid_0's binary_logloss: 0.376378
[17]	valid_0's binary_logloss: 0.369567
[18]	valid_0's binary_logloss: 0.366851
[19]	valid_0's binary_logloss: 0.366295
[20]	valid_0's binary_logloss: 0.365599
[21]	valid_0's binary_logloss: 0.364225
[22]	valid_0's binary_logloss: 0.363502
[23]	valid_0's binary_logloss: 0.363138
[24]	valid_0's binary_logloss:

In [12]:
clf = []
val_pred = pd.DataFrame([])
val_mcc = []
lgb_params = {
               'feature_fraction': 0.8,
               'metric': 'binary_logloss',
               'nthread':8, 
               'learning_rate': 0.1, 
               'objective': 'binary', 
               'num_leaves': 2**4,
               'verbose':0, 
               'seed':123
              }


for train_idx, val_idx in skf.split(train, target):
    X_tr, y_tr = train.iloc[train_idx, :], target[train_idx]
    X_val, y_val = train.iloc[val_idx, :], target[val_idx]
 
    model_lgb = lgb.train(lgb_params, lgb.Dataset(X_tr, label=y_tr), 500,\
                           valid_sets=lgb.Dataset(X_val, label=y_val), early_stopping_rounds=30)
    pred_lgb = model_lgb.predict(X_val)
    pred_lgb = pd.DataFrame(pred_lgb, index=val_idx)
    
    val_pred = pd.concat([val_pred, pred_lgb], axis=1)
    val_mcc.append(matthews_corrcoef(y_val, (pred_lgb>0.5).astype(int)))
    clf.append(model_lgb)

[1]	valid_0's binary_logloss: 0.193931
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's binary_logloss: 0.181205
[3]	valid_0's binary_logloss: 0.171893
[4]	valid_0's binary_logloss: 0.164794
[5]	valid_0's binary_logloss: 0.158944
[6]	valid_0's binary_logloss: 0.154606
[7]	valid_0's binary_logloss: 0.151593
[8]	valid_0's binary_logloss: 0.148168
[9]	valid_0's binary_logloss: 0.14553
[10]	valid_0's binary_logloss: 0.143149
[11]	valid_0's binary_logloss: 0.140537
[12]	valid_0's binary_logloss: 0.138668
[13]	valid_0's binary_logloss: 0.137481
[14]	valid_0's binary_logloss: 0.136696
[15]	valid_0's binary_logloss: 0.135746
[16]	valid_0's binary_logloss: 0.13459
[17]	valid_0's binary_logloss: 0.132824
[18]	valid_0's binary_logloss: 0.131348
[19]	valid_0's binary_logloss: 0.13074
[20]	valid_0's binary_logloss: 0.130459
[21]	valid_0's binary_logloss: 0.130309
[22]	valid_0's binary_logloss: 0.130155
[23]	valid_0's binary_logloss: 0.12973
[24]	valid_0's binary_logloss: 

[116]	valid_0's binary_logloss: 0.128302
[117]	valid_0's binary_logloss: 0.128638
[118]	valid_0's binary_logloss: 0.128967
[119]	valid_0's binary_logloss: 0.128764
[120]	valid_0's binary_logloss: 0.128709
[121]	valid_0's binary_logloss: 0.128985
[122]	valid_0's binary_logloss: 0.12894
[123]	valid_0's binary_logloss: 0.129115
[124]	valid_0's binary_logloss: 0.129188
[125]	valid_0's binary_logloss: 0.129421
[126]	valid_0's binary_logloss: 0.129468
[127]	valid_0's binary_logloss: 0.129255
[128]	valid_0's binary_logloss: 0.129282
[129]	valid_0's binary_logloss: 0.129065
[130]	valid_0's binary_logloss: 0.129532
[131]	valid_0's binary_logloss: 0.129581
[132]	valid_0's binary_logloss: 0.129644
[133]	valid_0's binary_logloss: 0.129962
[134]	valid_0's binary_logloss: 0.129936
[135]	valid_0's binary_logloss: 0.129694
[136]	valid_0's binary_logloss: 0.129715
[137]	valid_0's binary_logloss: 0.129927
[138]	valid_0's binary_logloss: 0.12979
[139]	valid_0's binary_logloss: 0.12976
[140]	valid_0's bin

In [13]:
pred_val = (np.sum(val_pred, axis=1)>0.5).astype(int)

In [14]:
comparison = pd.DataFrame({'target':target, 'pred_val':pred_val}) 
comparison[(comparison['target']==1 )&(comparison['pred_val']==1)].count()

target      221
pred_val    221
dtype: int64

In [15]:
comparison[(comparison['target']==0 )&(comparison['pred_val']==1)].count()

target      93
pred_val    93
dtype: int64

In [16]:
from sklearn.metrics import confusion_matrix

confusion_matrix(target, pred_val)

array([[8094,   93],
       [ 304,  221]])

In [17]:
pd.concat([train, comparison], axis=1)

,id_measurement,phase,NumPeaks,NumPosPeaks,NumNegPeaks,MaxWidth,MinWidth,MeanWidth,MaxAmp,MinAmp,MeanAmp,StdAmpAll,StdPosAll,StdWidthAll,StdAmpQ2,StdPosQ2,StdWidthQ2,StdAmpQ3,StdPosQ3,StdWidthQ3,StdAmpQ4,StdPosQ4,StdWidthQ4,target,pred_val
0,0,0,105,48,57,41741,1,3676.451923,46.676929,-21.931899,-1.251672e-04,11.866878,77424.493902,7099.857490,11.214203,25011.751218,6165.933632,12.479814,22686.469829,2801.657114,8.081718,22092.125444,8984.882428,0,0
1,0,1,23,14,9,69734,1,10977.954545,13.454715,-12.666357,9.360705e-05,8.663170,76017.661321,16116.212440,9.545417,8608.000000,0.000000,8.286657,19963.709475,6986.037612,5.719954,15027.735166,9284.827461,0,0
2,0,2,121,67,54,58988,1,3175.458333,26.176798,-30.415865,1.122479e-04,11.285462,74713.050192,7570.433485,14.920124,24129.371090,7265.204158,11.041417,20894.197560,1152.704984,11.427326,21303.210731,5779.776683,0,0
3,1,0,217,99,118,44999,1,1820.212963,91.147620,-38.193195,2.388918e-04,11.473998,131660.062871,4198.542517,5.535926,25928.722290,12482.994099,5.538974,22936.531117,3615.659712,5.086507,34060.095196,2594.629391,1,0
4,1,1,141,68,73,37017,1,2803.257143,14.246623,-9.247347,-6.453114e-05,4.289404,126355.390058,5433.900865,4.720264,21722.232143,7837.139326,3.000449,25632.115546,9773.675906,3.425179,29998.890319,4142.901139,1,0
5,1,2,269,136,133,34192,1,1479.992537,29.666366,-38.160564,-2.017077e-04,8.664104,136835.904736,3703.327258,7.840436,26726.993869,11064.686006,7.663944,22528.505946,2933.473694,5.424385,32707.960897,2488.802766,1,0
6,2,0,8,4,4,149204,1,26533.142857,20.490244,-24.061076,2.072749e-06,16.710737,73841.032761,50714.856497,6.776206,0.500000,0.000000,2.811011,2.000000,0.000000,NaN,NaN,NaN,0,0
7,2,1,10,4,6,82129,2,29769.333333,12.784146,-14.087355,-5.827755e-05,10.210538,97654.929316,23754.806831,7.766155,26663.500000,0.000000,10.971364,21282.297329,8127.196411,NaN,NaN,NaN,0,0
8,2,2,14,8,6,121243,1,23939.307692,24.047800,-21.953692,8.683281e-05,16.612100,109468.856841,32739.842097,0.000000,0.000000,NaN,18.213761,26874.597364,12570.099550,0.000000,0.000000,NaN,0,0
9,3,0,26,7,19,130989,1,14956.320000,9.922842,-9.190952,-2.328314e-04,6.866477,93420.407809,34586.008420,0.000000,0.000000,NaN,7.070540,10809.000000,0.000000,0.000000,0.000000,NaN,0,0


In [18]:
import scipy
print(scipy.__version__)
print(np.__version__)

1.1.0
1.14.3


In [19]:
import sklearn
print(sklearn.__version__)

0.20.2


In [20]:
from imblearn.under_sampling import RandomUnderSampler

In [22]:
sampler = RandomUnderSampler(random_state=42)
train_re, target_re = sampler.fit_resample(train.fillna(0), target)

In [24]:
train_re.shape

(1050, 23)

In [31]:
pd.Series(target_re).value_counts()

1    525
0    525
dtype: int64

In [33]:
clf = []
val_pred = pd.DataFrame([])
val_mcc = []
lgb_params = {
               'feature_fraction': 0.8,
               'metric': 'binary_logloss',
               'nthread':8, 
               'learning_rate': 0.1, 
               'objective': 'binary', 
               'num_leaves': 2**4,
               'verbose':0, 
               'seed':123
              }

for train_idx, val_idx in skf.split(train_re, target_re):
    X_tr, y_tr = train_re[train_idx, :], target_re[train_idx]
    X_val, y_val = train_re[val_idx, :], target_re[val_idx]
 
    model_lgb = lgb.train(lgb_params, lgb.Dataset(X_tr, label=y_tr), 500,\
                           valid_sets=lgb.Dataset(X_val, label=y_val), early_stopping_rounds=30)
    pred_lgb = model_lgb.predict(X_val)
    pred_lgb = pd.DataFrame(pred_lgb, index=val_idx)
    
    val_pred = pd.concat([val_pred, pred_lgb], axis=1)
    val_mcc.append(matthews_corrcoef(y_val, (pred_lgb>0.5).astype(int)))
    clf.append(model_lgb)

[1]	valid_0's binary_logloss: 0.638708
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's binary_logloss: 0.593258
[3]	valid_0's binary_logloss: 0.556483
[4]	valid_0's binary_logloss: 0.526723
[5]	valid_0's binary_logloss: 0.501264
[6]	valid_0's binary_logloss: 0.481347
[7]	valid_0's binary_logloss: 0.46333
[8]	valid_0's binary_logloss: 0.448833
[9]	valid_0's binary_logloss: 0.433952
[10]	valid_0's binary_logloss: 0.420137
[11]	valid_0's binary_logloss: 0.41113
[12]	valid_0's binary_logloss: 0.404058
[13]	valid_0's binary_logloss: 0.394153
[14]	valid_0's binary_logloss: 0.386149
[15]	valid_0's binary_logloss: 0.38148
[16]	valid_0's binary_logloss: 0.376378
[17]	valid_0's binary_logloss: 0.369567
[18]	valid_0's binary_logloss: 0.366851
[19]	valid_0's binary_logloss: 0.366295
[20]	valid_0's binary_logloss: 0.365599
[21]	valid_0's binary_logloss: 0.364225
[22]	valid_0's binary_logloss: 0.363502
[23]	valid_0's binary_logloss: 0.363138
[24]	valid_0's binary_logloss:

In [34]:
val_mcc

[0.7353762340761378,
 0.7252753706239207,
 0.6813774634083654,
 0.7866263316913054]

In [42]:
pred_train = pd.DataFrame([])
for model in clf:
    pred_train = pd.concat([pred_train, pd.DataFrame(model.predict(train)/4)], axis=1)

In [49]:
(np.sum(pred_train, axis=1)>0.5).astype(int).value_counts()

0    7092
1    1620
dtype: int64

In [50]:
confusion_matrix(target, (np.sum(pred_train, axis=1)>0.5).astype(int))

array([[7075, 1112],
       [  17,  508]])

In [51]:
feature_imp = np.zeros(train.shape[1])
for model in clf:
    feature_imp+=model.feature_importance(importance_type='gain')

pd.DataFrame(feature_imp/4, index=train.columns).sort_values(by=0, ascending=False)

,0
NumNegPeaks,1338.422381
NumPeaks,782.469207
NumPosPeaks,416.664809
MinAmp,316.016967
MaxAmp,233.370769
StdAmpQ4,205.395338
phase,170.230378
StdAmpAll,135.609225
id_measurement,115.642554
MeanWidth,108.234595


In [62]:
negtar = pd.concat([train['NumNegPeaks'], pd.Series(target)], axis=1)
negtar[negtar[0]==1]

,NumNegPeaks,0
3,118,1
4,73,1
5,133,1
201,98,1
202,15,1
228,0,1
229,8,1
230,50,1
270,90,1
271,16,1


In [64]:
train.iloc[2322:2346, :]

,id_measurement,phase,NumPeaks,NumPosPeaks,NumNegPeaks,MaxWidth,MinWidth,MeanWidth,MaxAmp,MinAmp,MeanAmp,StdAmpAll,StdPosAll,StdWidthAll,StdAmpQ2,StdPosQ2,StdWidthQ2,StdAmpQ3,StdPosQ3,StdWidthQ3,StdAmpQ4,StdPosQ4,StdWidthQ4
2322,774,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2323,774,1,1,0,1,0,0,0.000000,0.000000,-30.441270,-0.000076,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN
2324,774,2,2,2,0,1,1,1.000000,19.084034,0.000000,0.000090,1.048194,0.500000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.048194,0.500000,0.000000
2325,775,0,16,8,8,130520,1,23989.533333,21.210753,-51.611196,-0.000155,20.695580,116842.460720,33895.500732,0.000000,0.000000,NaN,15.109178,21660.259699,13113.380835,22.042148,18901.750056,17818.383821
2326,775,1,11,9,2,106074,2,35954.700000,64.383449,-72.162144,0.000221,40.747387,131386.717630,35971.917138,0.000000,0.000000,NaN,0.000000,0.000000,NaN,20.490371,26717.511788,14073.834573
2327,775,2,39,19,20,50594,1,9141.526316,34.647800,-29.679071,-0.000059,18.841521,109997.427269,14695.650794,17.927309,31581.430885,15545.042111,19.319627,21391.966408,7919.083548,17.811874,25196.371480,13819.290148
2328,776,0,23,12,11,139127,1,17486.863636,29.719951,-31.171995,0.000023,19.785435,88319.204753,36655.465444,0.000000,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.000000,NaN
2329,776,1,1,0,1,0,0,0.000000,0.000000,-13.396848,-0.000033,0.000000,0.000000,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2330,776,2,9,3,6,16111,1,4592.500000,36.055590,-26.889329,-0.000167,24.293204,13731.811663,5656.798808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2331,777,0,2,2,0,85,85,85.000000,20.531934,0.000000,0.000071,6.375201,42.500000,0.000000,NaN,NaN,NaN,6.375201,42.500000,0.000000,NaN,NaN,NaN


In [14]:
numpeaks = np.load('../features/numpeaks_test.npy')
numpospeaks = np.load('../features/numpospeaks_test.npy')
numnegpeaks = np.load('../features/numnegpeaks_test.npy')

maxpeakwidth = np.load('../features/maxpeakwidth_test.npy')
minpeakwidth = np.load('../features/minpeakwidth_test.npy')
meanpeakwidth = np.load('../features/meanpeakwidth_test.npy')
maxamp = np.load('../features/maxamp_test.npy')
minamp = np.load('../features/minamp_test.npy')
meanamp = np.load('../features/meanamp_test.npy')

stdampall = np.load('../features/stdampall_test.npy')
stdposall = np.load('../features/stdposall_test.npy')
stdwidthall = np.load('../features/stdwidthall_test.npy')

stdampq2 = np.load('../features/stdampq2_test.npy')
stdposq2 = np.load('../features/stdposq2_test.npy')
stdwidthq2 = np.load('../features/stdwidthq2_test.npy')
stdampq3 = np.load('../features/stdampq3_test.npy')
stdposq3 = np.load('../features/stdposq3_test.npy')
stdwidthq3 = np.load('../features/stdwidthq3_test.npy')
stdampq4 = np.load('../features/stdampq4_test.npy')
stdposq4 = np.load('../features/stdposq4_test.npy')
stdwidthq4 = np.load('../features/stdwidthq4_test.npy')


test_features = pd.DataFrame({'NumPeaks':numpeaks, 'NumPosPeaks':numpospeaks, 'NumNegPeaks':numnegpeaks, 'MaxWidth':maxpeakwidth,
                         'MinWidth':minpeakwidth, 'MeanWidth':meanpeakwidth, 'MaxAmp':maxamp, 'MinAmp':minamp, 'MeanAmp':meanamp,
                        'StdAmpAll':stdampall, 'StdPosAll':stdposall, 'StdWidthAll':stdwidthall, 'StdAmpQ2':stdampq2, 'StdPosQ2':stdposq2,
                         'StdWidthQ2':stdwidthq2, 'StdAmpQ3':stdampq3, 'StdPosQ3':stdposq3, 'StdWidthQ3':stdwidthq3,
                        'StdAmpQ4':stdampq4, 'StdPosQ4':stdposq4, 'StdWidthQ4':stdwidthq4,})

In [15]:
test = pd.concat([meta_test, test_features], axis=1)

In [18]:
test_id = test['signal_id'].values
del test['signal_id']

In [39]:
preds = None
for model in clf:
    if preds is None:
        preds = model.predict(test)
    else:
        preds += model.predict(test)
    
preds = preds/4

In [40]:
submission = pd.DataFrame({'signal_id':test_id,'target':(preds>0.5).astype(int)})

In [44]:
submission.to_csv('../output/submission_1.csv', index=False)